In [ ]:
import fiona
from shapely.geometry import shape
import rtree
import pyprind

In [ ]:
sonnendach = "solar_new_properties_added.geojson"
parcels = "parcels.gpkg"

In [ ]:
rtree_index = rtree.Rtree()

with fiona.open(parcels, 'r') as source:
    for index, feature in enumerate(source):
        feature['geometry'] = shape(feature['geometry'])
        rtree_index.add(index, feature['geometry'].bounds, obj=feature)

In [ ]:
with fiona.open(sonnendach, 'r') as source:
    sink_schema = source.schema
    sink_schema['properties']['parcel_id'] = 'int'

    with fiona.open(
            'sd_linked_parcels.gpkg', 'w',
            crs=source.crs,
            driver="GPKG",
            schema=sink_schema,
            ) as sink:

        for i, feature in enumerate(source):
            if not i % 100000:
                print(i)
            geom = shape(feature['geometry'])
            for o in rtree_index.intersection(geom.bounds, objects=True):
                parcel = o.object
                if geom.intersects(parcel['geometry']):
                    feature['properties']['parcel_id'] = int(parcel['properties']['ID1'])
                    # Just use first one, good enough
                    sink.write(feature)
                    break